In [1]:
import requests
import random
from transformers import pipeline

/Users/nandinibohra/Desktop/VSCodeFiles/CatGPT/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Going with J-Hartmann's DistilRoBERTa model for sentiment classification
emotion_model = pipeline("text-classification", 
                         model="j-hartmann/emotion-english-distilroberta-base", 
                         return_all_scores=True)

Device set to use cpu
/Users/nandinibohra/Desktop/VSCodeFiles/CatGPT/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [3]:
# Up-sampling the emotions + adding adjectives for diverse searches:
emotion_labels_upsampled = {
    "anger": ["angry", "furious", "mad", "annoyed", "irritated", "upset"],
    "disgust": ["gross", "ew", "grossed out", "puke"],
    "fear": ["scared", "worried", "terrified", "anxious"],
    "joy": ["happy", "excited", "joy", "dancing", "woohoo","yay", "yippee", "cute"],
    "neutral": ["neutral", "sleeping", "cool", "alright", "nice", "chill"],
    "sadness": ["sad", "unhappy", "crying", "sobbing", "lonely"],
    "surprise": ["surprised", "shocked", "startled"]
}

In [62]:
text = "I'm soooo tired."

label = emotion_model(text)[0]
label = max(label, key=lambda x: x['score'])['label']
print(label)

label_synonyms = emotion_labels_upsampled.get(label, ["cat"]) # find synonyms of label, else return default cat
queries = [f"{s} cat" for s in random.sample(label_synonyms, min(2, len(label_synonyms)))]
print(queries)

sadness
['sad cat', 'unhappy cat']


In [63]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [64]:
def fetch_tenor_gifs(emotion, limit=15):
    tenor_key = os.getenv("TENOR_KEY")
    search_terms = [f"{emotion} cat"]
    results = []
    for term in search_terms:
        url = f"https://tenor.googleapis.com/v2/search?q={term}&key={tenor_key}&limit={limit}&contentfilter=medium"
        res = requests.get(url).json()
        # print(res)
        results.extend(res.get("results", []))

        # results += [r['media_formats']['gif']['url'] for r in res['results']]
    return results

In [ ]:
import os
import requests

def fetch_tenor_gifs(emotion, limit=15, timeout=10):
    """Fetch Tenor GIF results for an emotion (returns list of result dicts).
    Checks HTTP status before parsing JSON and handles errors cleanly.
    """
    tenor_key = os.getenv("TENOR_KEY")
    if not tenor_key:
        print("TENOR_KEY not set. Set it in your environment or .env file.")
        return []

    search_terms = [f"{emotion} cat"]
    results = []

    for term in search_terms:
        url = f"https://tenor.googleapis.com/v2/search?q={term}&key={tenor_key}&limit={limit}&contentfilter=medium"
        try:
            resp = requests.get(url, timeout=timeout)
            resp.raise_for_status()  # raises HTTPError for 4xx/5xx
        except requests.exceptions.HTTPError as e:
            # Server returned an HTTP error (4xx/5xx) — skip this term
            print(f"HTTP error fetching '{term}': {e}")
            continue
        except requests.RequestException as e:
            # Network-level errors (connection, timeout, DNS, etc.)
            print(f"Request failed for '{term}': {e}")
            continue

        # Only now parse JSON safely
        try:
            data = resp.json()
        except ValueError:
            print(f"Invalid JSON response for '{term}'")
            continue

        # Merge results (use .get to avoid KeyError)
        results.extend(data.get("results", []))

    return results

# Example usage (run this in a notebook cell after setting TENOR_KEY):
# gifs = fetch_tenor_gifs("sad", limit=10)
# print(f"Found {len(gifs)} results")
# for g in gifs[:5]:
#     print(g.get("media_formats", {}).get("gif", {}).get("url"))

In [65]:
def filter_tenor_results(results, emotion_keywords):
    filtered = []
    allowed_cats = {"cat", "kitty", "kitten"}
    banned_words = {"kiss", "dog", "puppy", "baby", "human", "person"}

    for r in results:
        desc = r.get("content_description", "").lower()
        tags = [t.lower() for t in r.get("tags", [])]

        # Check if it's actually cat-related
        is_cat = any(cat in tags or cat in desc for cat in allowed_cats)

        # Check if emotion roughly matches
        matches_emotion = any(e in desc or e in tags for e in emotion_keywords)

        # Avoid irrelevant ones
        has_banned = any(b in desc or b in tags for b in banned_words)

        if is_cat and matches_emotion and not has_banned:
            filtered.append(r)

    return filtered

In [66]:
for query in queries:
    results = fetch_tenor_gifs(query)
    emotion_kywds = label_synonyms
    gifs = filter_tenor_results(results, emotion_kywds)
    print(f"GIFs for query '{query}':")

    # change toe filtered gifs if needed, filtering WIP for now
    for gif in gifs:
        print(gif["media_formats"]["gif"]["url"])
    print("-----------------------------------------------------------------------")

GIFs for query 'sad cat':
https://media.tenor.com/l3TkHdtrnV8AAAAC/puppy-dog-eyes-sad.gif
https://media.tenor.com/LZ_NgKi0JesAAAAC/cat-sad.gif
https://media.tenor.com/qlwiQMzHCV8AAAAC/sigh-le.gif
https://media.tenor.com/mId08vrW5WgAAAAC/sigh-le.gif
https://media.tenor.com/eEjOl43x05YAAAAC/cat-crying.gif
-----------------------------------------------------------------------
GIFs for query 'unhappy cat':
https://media.tenor.com/LZ_NgKi0JesAAAAC/cat-sad.gif
https://media.tenor.com/l3TkHdtrnV8AAAAC/puppy-dog-eyes-sad.gif
https://media.tenor.com/qlwiQMzHCV8AAAAC/sigh-le.gif
https://media.tenor.com/mId08vrW5WgAAAAC/sigh-le.gif
-----------------------------------------------------------------------


In [67]:
chosen_gif = random.choice(results)
print("Chosen GIF URL:", chosen_gif["media_formats"]["gif"]["url"])

Chosen GIF URL: https://media.tenor.com/qlwiQMzHCV8AAAAC/sigh-le.gif
